In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import seaborn as sns

In [75]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Q1

In [52]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [53]:
data = pd.DataFrame(data)

In [56]:
data

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [55]:
data = data[data['Borough'] != 'Not assigned'].reset_index()

In [57]:
data = data.drop('index',axis=1)

In [58]:
data

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [61]:
data.shape

(103, 3)

# Q2

In [65]:
geospatial_url = "https://cocl.us/Geospatial_data"
geo_data = pd.read_csv(geospatial_url)

In [66]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
data = data.merge(geo_data, on='Postal Code')

In [72]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Coordinates
0,M3A,North York,Parkwoods,43.753259,-79.329656,"(43.7532586, -79.3296565)"
1,M4A,North York,Victoria Village,43.725882,-79.315572,"(43.725882299999995, -79.31557159999998)"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"(43.6542599, -79.3606359)"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"(43.718517999999996, -79.46476329999999)"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"(43.6623015, -79.3894938)"


# Q3

In [70]:
data['Coordinates'] = list(zip(data['Latitude'],data['Longitude']))

In [73]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Coordinates
0,M3A,North York,Parkwoods,43.753259,-79.329656,"(43.7532586, -79.3296565)"
1,M4A,North York,Victoria Village,43.725882,-79.315572,"(43.725882299999995, -79.31557159999998)"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"(43.6542599, -79.3606359)"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"(43.718517999999996, -79.46476329999999)"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"(43.6623015, -79.3894938)"


In [93]:
map_toronto = folium.Map(location=[43.654260, -79.38949], zoom_start=10)

for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto